# 01 Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import os

# 02 Importing Data

In [8]:
path=r'/Users/serenakunzmann/Documents/CareerFoundry-Python/03-2025 Instacart Basket Analysis/Data/Prepared Data'

In [50]:
df_prods=pd.read_csv(os.path.join(path,'products_wrangled.csv'),index_col=False)

In [158]:
df_ords=pd.read_csv(os.path.join(path,'orders_wrangled.csv'),index_col=False)

In [18]:
df_dep=pd.read_csv(os.path.join(path,'departments_wrangled.csv'),index_col=False)

# 03 Data Consistency

# 3.0 Data Consistency Check - test dataframe

In [20]:
#create a dataframe
df_test=pd.DataFrame()

In [24]:
#create a mixed type column
df_test['mix']=['a','b',1, True]

In [26]:
df_test.head()

,mix
0,a
1,b
2,1
3,True


In [132]:
#check for mixed type columns
for col in df_test.columns.tolist():
  weird = (df_test[[col]].map(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

In [86]:
#reassigning mix type columns to one uniform type
df_test['mix'] = df_test['mix'].astype('str')

# 3.1 Data Consistency Checks - products dataframe

In [88]:
#checking for missing data
df_prods.isnull().sum()

product_id       0
product_name     0
aisle_id         0
department_id    0
prices           0
dtype: int64

In [92]:
#getting closer look at missing data
df_nan=df_prods[df_prods['product_name'].isnull()==True]

In [40]:
df_nan

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
33,33,34,NaN,121,14,12.2
68,68,69,NaN,26,7,11.8
115,115,116,NaN,93,3,10.8
261,261,262,NaN,110,13,12.1
525,525,525,NaN,109,11,1.2
1511,1511,1511,NaN,84,16,14.3
1780,1780,1780,NaN,126,11,12.3
2240,2240,2240,NaN,52,1,14.2
2586,2586,2586,NaN,104,13,12.4
3159,3159,3159,NaN,126,11,13.1


In [52]:
df_prods.shape

(49693, 5)

In [74]:
#removing missing data
df_prods_clean=df_prods[df_prods['product_name'].isnull()==False]

In [56]:
df_prods_clean.shape

(49677, 5)

In [76]:
#dropping missing data 
df_prods.dropna(inplace=True)

In [60]:
df_prods.shape

(49677, 5)

In [80]:
#identifying duplicates
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [64]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [78]:
#removing duplicates
df_prods_clean_no_dups=df_prods_clean.drop_duplicates()

In [68]:
df_prods_clean_no_dups.shape

(49672, 5)

In [102]:
#running exploratory data analysis on products clean no dups dataframe
df_prods_clean_no_dups.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,9.993282
std,14340.705287,38.315784,5.850779,453.615536
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,99999.000000


Observations: prices column looks off. The mean is 9.99 and max 99999.0. This indicates there are extreme outliers and this needs further investigation

In [106]:
# identifying outliers by calling out prices higher than the mean + 1 STD.
df_prods_clean_no_dups[df_prods_clean_no_dups['prices'] > df_prods['prices'].mean() * (df_prods['prices'].std())]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


In [108]:
# replacing outlier prices with mean price and saving to a new dataframe
df_prods_fixed=df_prods_clean_no_dups.replace({"prices":{14900.0:7.1,99999.0:7.1}})

In [110]:
# checking df_prods_fixed prices
df_prods_fixed.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,7.680414
std,14340.705287,38.315784,5.850779,4.199298
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,25.000000


In [82]:
#exporting checked products data
df_prods_clean_no_dups.to_csv(os.path.join(path, 'products_checked.csv'))

In [112]:
#exporting checked & fixed products data
df_prods_fixed.to_csv(os.path.join(path,'products_checked_fixed.csv'))

# 3.2 Data Consistency Check - orders dataframe

In [160]:
df_ords.describe()

,order_id,user_id,order_number,Order_day_of_week,order_hour_of_day,days_since_prior_order
count,9.999990e+05,999999.000000,999999.000000,999999.000000,999999.000000,939921.000000
mean,1.711274e+06,30105.433949,17.193345,2.779240,13.449694,11.097082
std,9.876569e+05,17323.577936,17.716800,2.048188,4.220945,9.183694
min,6.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000
25%,8.557595e+05,15117.000000,5.000000,1.000000,10.000000,4.000000
50%,1.711006e+06,30148.000000,11.000000,3.000000,13.000000,7.000000
75%,2.567507e+06,45132.000000,24.000000,5.000000,16.000000,15.000000
max,3.421083e+06,60078.000000,100.000000,6.000000,23.000000,30.000000


Observation: The accuracy of columns above look in tact aside from days_since_prior_order. This columns total count is less than total counts for the other columns. This is likely due to missing values. Further investigation is needed.

In [162]:
#checking for mixed type data in orders dataframe
for col in df_ords.columns.tolist(): 
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1) 
  if len (df_ords[weird]) > 0: 
    print (col) 

/var/folders/dc/m090wqld5fs_bfy3ppvjgp380000gn/T/ipykernel_11358/1028877522.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
/var/folders/dc/m090wqld5fs_bfy3ppvjgp380000gn/T/ipykernel_11358/1028877522.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
/var/folders/dc/m090wqld5fs_bfy3ppvjgp380000gn/T/ipykernel_11358/1028877522.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
/var/folders/dc/m090wqld5fs_bfy3ppvjgp380000gn/T/ipykernel_11358/1028877522.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].ilo

In [164]:
#checking another way if there are mixed data types
df_ords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                999999 non-null  int64  
 1   user_id                 999999 non-null  int64  
 2   order_number            999999 non-null  int64  
 3   Order_day_of_week       999999 non-null  int64  
 4   order_hour_of_day       999999 non-null  int64  
 5   days_since_prior_order  939921 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 45.8 MB


Observation: No mixed type data found. 

In [166]:
#checking for missing data in orders dataframe
df_ords.isnull().sum()

order_id                      0
user_id                       0
order_number                  0
Order_day_of_week             0
order_hour_of_day             0
days_since_prior_order    60078
dtype: int64

In [168]:
df_nan=df_ords[df_ords['days_since_prior_order'].isnull()== True]

In [ ]:
df_ords=df_

In [170]:
df_nan

,order_id,user_id,order_number,Order_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
11,2168274,2,1,2,11,NaN
26,1374495,3,1,1,14,NaN
39,3343014,4,1,6,11,NaN
45,2717275,5,1,3,12,NaN
...,...,...,...,...,...,...
999917,1323107,60074,1,1,14,NaN
999927,3219075,60075,1,1,15,NaN
999945,1789638,60076,1,6,8,NaN
999960,1004677,60077,1,1,12,NaN


Observation: 60078 missing values were found in days_since_prior_order. This makes sense because they are first time customers so they dont have a prior order. We will leave these missing values in.

In [172]:
#checking for dups
df_ords[df_ords.duplicated()]

,order_id,user_id,order_number,Order_day_of_week,order_hour_of_day,days_since_prior_order


Observation: No duplicates were found.

In [180]:
#exporting orders dataframe
df_ords_clean=df_ords
df_ords_clean.to_csv(os.path.join(path,'orders_checked.csv'))